In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder, MinMaxScaler
import ast

In [2]:
# Cargar solo los datos necesarios
movies = pd.read_csv(r'../Datasets/transformed_movies.csv')
movies_reduced = movies[['title', 'popularity', 'collection', 'genres_encoded']].copy()

In [ ]:
# Rellenar valores nulos en la columna de colecciones
movies_reduced['collection'] = movies_reduced['collection'].fillna('No Collection')

In [ ]:
# Asegurarse de que los géneros estén en formato de lista
def process_genres(genre_str):
    try:
        return ast.literal_eval(genre_str)
    except ValueError:
        return None  # Devolver None si hay un error

In [ ]:
# Aplicar la función y eliminar registros donde genres_encoded es inválido
movies_reduced['genres_encoded'] = movies_reduced['genres_encoded'].apply(process_genres)

In [ ]:
# Eliminar filas con valores None en 'genres_encoded'
movies_reduced = movies_reduced[
    movies_reduced['genres_encoded'].notna() & 
    (movies_reduced['genres_encoded'].apply(len) > 0)
]

In [ ]:
# Procesar géneros usando MultiLabelBinarizer
mlb = MultiLabelBinarizer()
genres_matrix = mlb.fit_transform(movies_reduced['genres_encoded'])
movies_features = pd.DataFrame(genres_matrix, columns=mlb.classes_)

In [ ]:
# Procesar colecciones usando LabelEncoder
le = LabelEncoder()
movies_reduced['collection_encoded'] = le.fit_transform(movies_reduced['collection'])

# Asignar 0 a las películas sin colección ("No Collection")
movies_reduced.loc[movies_reduced['collection'] == 'No Collection', 'collection_encoded'] = 0

# Escalar las colecciones a un rango entre 0 y 1 usando MinMaxScaler
scaler = MinMaxScaler()
movies_reduced['collection_encoded_scaled'] = scaler.fit_transform(movies_reduced[['collection_encoded']])

# Combinar características: géneros y colecciones escaladas
movies_features['collection_scaled'] = movies_reduced['collection_encoded_scaled'].values

In [ ]:
# Agregar la popularidad original a movies_features
movies_features['popularity'] = movies_reduced['popularity'].values

# Normalizar la popularidad dentro de movies_features
scaler = MinMaxScaler()
movies_features['popularity_scaled'] = scaler.fit_transform(movies_features[['popularity']])

# Eliminar la columna original de popularidad
movies_features.drop(columns=['popularity'], inplace=True)

In [3]:
# Guardar la matriz combinada en un archivo comprimido
np.savez_compressed("../Datasets/movies_features.npz", movies_features.values)

# Guardar el DataFrame reducido como CSV
movies_reduced[['title', 'popularity', 'collection', 'genres_encoded']].to_csv(r'../Datasets/movies_reduced.csv', index=False)